In [ ]:
pip install -U spacy

In [ ]:
!python -m spacy download xx_sent_ud_sm

     |████████████████████████████████| 4.3 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('xx_sent_ud_sm')


# Importation & inititalizing

In [ ]:
import spacy 
from spacy.tokens import DocBin
from tqdm import tqdm
import json
import re

# Reading Dataset & Presenting over view

In [ ]:
with open("Dataset.json")as f :
    dic= json.load(f)
Classes=dic["classes"]
Annotations=dic["annotations"]
Diclass={item:0 for item in Classes}
print(f"Number of pragraphs :{len(dic['annotations'])}")
for item in Annotations:
    for subitem in item[1]["entities"]:
        Diclass[subitem[2]]+=1

Number of pragraphs :6081


In [ ]:
Diclass

{'CULTURE ET ART': 4775,
 'DIVERTISSEMENT': 380,
 'ECONOMIE': 4644,
 'EDUCATION': 4303,
 'GASTRONOMIE': 111,
 'MODE': 28,
 'MÉDIAS': 4621,
 'POLITIQUE': 4656,
 'RELIGION': 185,
 'SANTÉ': 4579,
 'SECURITÉ ET DEFENSE': 4410,
 'SPORT': 4663,
 'TECHNOLOGIE': 4508}

In [ ]:
def process(text):
    text = re.sub('[^\w]+|_',' ', text, flags=re.U)
    text=re.sub(r'[a-zA-Z?]','',text)
    text=re.sub(f'\n','',text)
    text=re.sub(f'\t','',text)
    text=re.sub('\'','',text)
    text=re.sub('\"','',text)
    text =re.sub(" +"," ",text)
    return text
def preprocess(text):
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    return text

# Creating Spacy Binary for training

In [ ]:
nlp=spacy.blank("ar")
db=DocBin()
for text, annot in tqdm(Annotations): # data in previous format
    text=preprocess(text)
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            if not(label in ["RELIGION","GASTRONOMIE","MODE","DIVERTISSEMENT"]):
              ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

#db.to_disk(input("OUTPUT NAME (Without Extension):")+".spacy")
print("Wrote to Disk Succesfully")

100%|██████████| 6081/6081 [00:07<00:00, 809.67it/s]

Wrote to Disk Succesfully


# Loading The Spacy Binary 

In [ ]:
docs=[]
docs = list(db.get_docs(nlp.vocab))

In [ ]:
len(docs)

6081

# Creating Training & Test Data

In [ ]:
import random
random.shuffle(docs)
val=int(0.8*len(docs))
random.shuffle(docs)
train_docs = docs[:val]
dev_docs = docs[val:]
# Create and save a collection of training docs
train_docbin = DocBin(docs=train_docs)
train_docbin.to_disk("train.spacy")
# Create and save a collection of evaluation docs
dev_docbin = DocBin(docs=dev_docs)
dev_docbin.to_disk("dev.spacy")

# Training process

## Creating the source of the truth (config.cfg)

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


## Starting The training

In [ ]:
!python -m spacy train config.cfg --output ./output1 --paths.train ./train.spacy --paths.dev ./dev.spacy 

ℹ Saving to output directory: output1
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-05-19 07:42:16,081] [INFO] Set up nlp object from config
[2022-05-19 07:42:16,093] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-19 07:42:16,097] [INFO] Created vocabulary
[2022-05-19 07:42:16,099] [INFO] Finished initializing nlp object
[2022-05-19 07:42:23,381] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     48.76    0.05    0.69    0.03    0.00
  0     200         73.38   5169.93    5.96   36.19    3.25    0.06
  0     400        332.02   5688.66    6.21   48.00    3.32    0.06
  0     600       1527.10   6745.38   10.7

In [ ]:
from google.colab import files
!zip -r output.zip output1

updating: output1/ (stored 0%)
updating: output1/model-last/ (stored 0%)
updating: output1/model-last/tokenizer (deflated 81%)
updating: output1/model-last/config.cfg (deflated 61%)
updating: output1/model-last/ner/ (stored 0%)
updating: output1/model-last/ner/moves (deflated 73%)
updating: output1/model-last/ner/cfg (deflated 33%)
updating: output1/model-last/ner/model (deflated 7%)
updating: output1/model-last/tok2vec/ (stored 0%)
updating: output1/model-last/tok2vec/cfg (stored 0%)
updating: output1/model-last/tok2vec/model (deflated 7%)
updating: output1/model-last/meta.json (deflated 63%)
updating: output1/model-last/vocab/ (stored 0%)
updating: output1/model-last/vocab/vectors (deflated 45%)
updating: output1/model-last/vocab/strings.json (deflated 90%)
updating: output1/model-last/vocab/lookups.bin (stored 0%)
updating: output1/model-last/vocab/vectors.cfg (stored 0%)
updating: output1/model-last/vocab/key2row (stored 0%)
updating: output1/model-best/ (stored 0%)
updating: outpu

# Examples 

In [ ]:
nlp=spacy.load("output1/model-best")
text="يعلم الحزب الدستوري الحر في بلاغ تلقينا نسخة منه أنه وجه محضر تنبيه بواسطة عدل تنفيذ للهيئة العليا المستقلة للإنتخابات في شخص ممثلها القانوني تضمن مطالبتها بالقيام بدورها في ضمان علوية القانون والامتناع عن تنظيم اي استفتاء أو انتخابات بناء على نصوص غير صادرة عن مؤسسة تشريعية منتخبة كالامتناع عن طلب أو صرف أي مبالغ مالية من ميزانية الدولة لتنظيم محطات انتخابية غير شرعية نص البلاغ اعتبارا لتسيير الدولة حاليا بموجب الفصل 80 الذي يسمح لمن فعله باتخاذ تدابير استثنائية تزول بزوال أسبابها وتهدف لاستئناف السير العادي لدواليب الدولة في أقرب الآجال ولا يمكن الحاكم من سن قوانين دائمة تحدد مصير البلاد وتمس من المعايير الدولية للمنظومة الانتخابية وبما أن القانون عدد 16 لسنة 2014 المؤرخ في 26 ماي 2014 المنقح في 2019 المنظم للانتخابات والاستفتاء ساري المفعول حاليا"
text = re.sub("[إأٱآا]", "ا", text)
text = re.sub("ى", "ي", text)
text = re.sub("ؤ", "ء", text)
text = re.sub("ئ", "ء", text)
text = re.sub("ة", "ه", text)
doc=nlp(text)
from spacy import displacy
displacy.render(doc,style="ent",jupyter=True)

In [ ]:
text="حيث تم تحديد يوم السبت 19 مارس 2022  موعدا لاستئناف نشاط البطولة فيما سيكون يوم 17 مارس الحالي موعدا لغلق الميركاتو و في ما يلي بلاغ الجامعة"
text = re.sub("[إأٱآا]", "ا", text)
text = re.sub("ى", "ي", text)
text = re.sub("ؤ", "ء", text)
text = re.sub("ئ", "ء", text)
text = re.sub("ة", "ه", text)
doc=nlp(text)
from spacy import displacy
displacy.render(doc,style="ent",jupyter=True)

In [ ]:
doc=nlp(preprocess("أفادت قناة سكاي نيوز الناطقة بالعربية نقلا عن وسائل إعلام روسية وأوكرانية بأن موعد الجولة الثانية من المفاوضات بين البلدين بهدف وقف العمليات العسكرية الروسية في أوكرانيا المستمرة منذ الخميس ستقام غدا الأربعاء 2 مارس 2022 بعد الجولة الأولى التي جرت أمس الإثنين على الحدود بين أوكرانيا وبيلاروسيا"))
doc1=nlp(preprocess("عبر الأمين العام لحزب التيار الديمقراطي غازي الشواشي اليوم الثلاثاء 01 مارس 2022  عن شفقته على حكومة نجلاء بودن     وفي حوار له في ماتينال اذاعة شمس انتقد الشواشي تعاطي وأداء الحكومة  وقال إن حكومة بودن هي حكومة صامتة لا تتكلم ولا تتفاعل مع كل المقترحات واعتبر غازي الشواشي أن سنة 2022 ستكون سنة التضخم بامتياز  خاصة في ضوء ارتفاع سعر برميل البترول والحرب في أوكرانيا  مبينا أن الزيادة في أسعار المحروقات التي تم الإعلان عنها ليلة أمس كانت منتظرة    وتحدث الشواشي عن ضرورة إيجاد سياسات واضحة للمالية العمومية والتوصل إلى التوازنات الضرورية وعبر عن أمله في تكون للحكومة القدرة في التقليص من حدة التضخم عبر وضع برنامج ورؤية واضحة  قائلا إن حكومة نجلاء بودن دون رؤية منذ يومها الأول تونس م هددة بالانهيار وفي مرحلة أخطر من 2013وحذر الشواشي من خطورة الأوضاع التي تمر بها تونس  مؤكدا أن البلاد في مرحلة أخطر من 2013 وفق تعبيره وأوضح الشواشي أن تونس تعيش اليوم حالة  من الشلل في ضوء التفرد بالرأي لرئيس الجمهورية وعجز حكومة نجلاء بودن وتحدث غازي الشواشي عن مشروع هلامي في تونس وعن عزلة داخلية وخارجية  مشددا على ضرورة تنظيم حوار والتوصل لتحقيق الإستقرار في البلاد وتابع أن الوضع في تونس قابل للإنفجار في أي لحظة وعبر المتحدث عن انفتاح حزبه على رئيس الجمهورية وقال إن أيديهم ممدودة له  معتبرا أنه لا أحد له وصاية على تونس وتونس لكل التونسيين وفق تصريحه  يدعو سعي د إلى إطلاق حوار وطنيودعا الشواشي  رئيس الجمهورية قيس سعيد إلى إطلاق حوار وطني والتوصل إلى خارطة طريق لإنقاذ تونس من الإنهيار واعتبر الشواشي أن الحوار ينقسم إلى مسارين  المسار السياسي بما يشمل قانون الأحزاب والنظام السياسي وإصلاح القضاء  أما المسار الثاني فهو المسار الإقتصادي وهو مسار حساس وقصير في زمن ولا يمكن الإنتظار وتحدث الشواشي عن أهمية تنظيم الحوار بإشراف رئيس الجمهورية قيس سعيد  لكن وفي صورة رفضه للحوار فيجب التوجه إلى حوار دون الرئيس وفق تعبيره وجدد غازي الشواشي التأكيد على أن قيس سعيد ليس له أي رؤية أو خيارات في المجال الإقتصادي   "))
doc2=nlp(preprocess("واوضح ان التطبيقة ستساهم في مزيد ربط الصلة بمختلف الفاعلين الثقافيين من جمعيات ومؤسسات ثقافية وفنانين ومتعهدي حفلات وستوفر لهم ارضية افتراضية للتفاعل الحيني بين الفاعلين والادارة بما يساهم في تطوير المشهد الثقافي الجهوي فضلا عن فسح المجال لتوثيق مختلف الانشطة والتظاهرات التي تنتظم بالجهة  ولاحظ في ذات السياق ان التطبيقة ستكون مفتوحة لكل المستعملين وستمكن المولعين بالثقافة والفنون من اجندا ثقافية لمختلف التظاهرات الثقافية المبرمجة بالجهة"))
displacy.render(doc2,style="ent",jupyter=True)
displacy.render(doc1,style="ent",jupyter=True)
displacy.render(doc,style="ent",jupyter=True)

In [ ]:
text="وذكر موقع داون ديتيكتور دوت كوم لمراقبة عمليات الانقطاع أن نحو 40 ألف مستخدم أبلغوا عن مشكلات في منصة التواصل الاجتماعي يوم الجمعة. وربما يكون الانقطاع قد أثر على عدد أكبر من المستخدمين."
doc12=nlp(preprocess(text))


In [ ]:
displacy.render(doc12,style="ent",jupyter=True)

In [ ]:
files.download("output.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Notebook End